In [ ]:
import pandas as pd
import numpy as np

Loading the raw contribution tree into Python

In [ ]:
loading_url = 'UseCase_Data/Apple_UseCase/rawExcel/TableApple1kg_ClimateChange.xlsx'
raw = pd.read_excel(loading_url)

Specifying an index, thereby creating a unique identifier with key semantics

In [ ]:
r_columns = raw.columns.to_list()
h_depth = r_columns.index('Process') #by looking where the process column is situated we can determine how deep the contribution tree is
raw['id'] = raw.index


Converting the indivual hierarchy level columns into a single attribute denoting the location of a process within the hierarchy

In [ ]:
hierarchy_levels = []
for row in range(len(raw)):
    for col in r_columns[:h_depth]:
        if not np.isnan(raw[col].iloc[row]):
            hierarchy_levels.append(r_columns.index(col))


In [ ]:
raw['hierarchy_level'] = hierarchy_levels
raw.drop(columns=r_columns[:h_depth], inplace = True) #dropping the old hierarchy level columns

Encoding the explicit child-parent relationships to create an adjancy list structure that stores the hierarchical information

In [ ]:
parents = []
for row in range(len(raw)):
    u_family = ['']
    for i in range(len(raw.iloc[:row+1])):
        if raw['hierarchy_level'].iloc[row] > raw['hierarchy_level'].iloc[i]:
            u_family.append(raw['id'].iloc[i])
    parents.append(u_family[-1])
raw['parent_id'] = parents
    

Adding dummy children processes to acount for parent processes which impact cannot be fully attributed to the children of that parent

In [ ]:
for i in range(len(raw)): #accounting for the fact that the openLCA contribution tree does not show the impact of output flows as processes
    if len(raw[raw.parent_id == raw.iloc[i].id]) > 0:
        raw = raw.append({
            "Process": raw.iloc[i].Process,
            'Amount': raw.iloc[i].Amount - sum(raw[raw['parent_id'] == raw.iloc[i].id].Amount),
            'Unit': raw.iloc[i].Unit,
            'id': 'temp',
            'hierarchy_level': raw.iloc[i].hierarchy_level + 1,
            'parent_id': raw.iloc[i].id
        }, ignore_index=True)
        

Filtering the data to exclude insignificant processes in terms of their contribution towards the overall impact

In [ ]:
percentage = 0.1
for row in range(len(raw)):
    if (raw['Amount'][row] / raw[raw['hierarchy_level'] == 0]['Amount'].values[0]) * 100 < percentage:
        raw.drop([row], inplace=True)
raw.reset_index(inplace=True)
raw['id'] = raw.index

Small fix to reset the index

In [ ]:
converted_parents = []
for i in range(len(raw)):
    if raw.iloc[i].parent_id != '':
        converted_parents.append(raw[raw['index'] == raw.iloc[i].parent_id].index[0])
    else:
        converted_parents.append('')
raw['parent_id'] = converted_parents
raw.drop(columns=['index'], inplace=True)

Extracting the life cycle phases from the process names into a seperate attribute and distributing that life cycle phase to all children 

In [ ]:
phases = []

#the lines of code below determine the lifecycle phases of processes, however,
#this is data specific and therefore hard coded for the use-cases of the thesis

#!!! for anyone using this script on custom data, read the comment above !!!
if 'Apple_UseCase' in loading_url:
    for row in range(len(raw)):
        if raw['hierarchy_level'].iloc[row] == 0:
            phases.append('root')
        elif raw['hierarchy_level'].iloc[row] == 1 and 'Farm' in raw['Process'].iloc[row]:
            phases.append('farm')
        elif raw['hierarchy_level'].iloc[row] == 1 and 'Distribution' in raw['Process'].iloc[row]:
            phases.append('distribution')
        elif raw['hierarchy_level'].iloc[row] == 1 and 'Retail' in raw['Process'].iloc[row]:
            phases.append('retail')
        elif raw['hierarchy_level'].iloc[row] == 1 and 'Consumption' in raw['Process'].iloc[row]:
            phases.append('consumption')
        else:
            phases.append('other')
if 'Chocolate_UseCase' in loading_url:
    for row in range(len(raw)):
        if raw['hierarchy_level'].iloc[row] == 0:
            phases.append('root')
        elif raw['hierarchy_level'].iloc[row] == 1:
            phases.append('processing')
        elif 'at farm' in raw['Process'].iloc[row]:
            phases.append('farm')
        else:
            phases.append('other')
if 'Pizza_UseCase' in loading_url:
    phases = ['raw materials/ingredients']*len(raw)
    
raw['Phase'] = phases

In [ ]:
for row in range(len(raw)):
    if raw['Phase'].iloc[row] == 'other':
        raw['Phase'][row] = raw[raw['id'] == raw['parent_id'].iloc[row]]['Phase'].values[0]

In [ ]:
raw

Extracting the region from the process names into a seperate attribute

In [ ]:
locations = pd.read_csv('locations.csv', delimiter=';') #importing the list of region-identifier combinations that was created manually beforehand
location_strings = [' - ' + str(code) for code in locations.Code.to_list()]
locations_dict = {}
for i in range(len(locations)):
    locations_dict[location_strings[i]] = locations.iloc[i].Name

In [ ]:
raw_location_list = []
for i in range(len(raw)):
    found = False
    for string in location_strings:
        if raw.iloc[i].Process.endswith(string):
            raw_location_list.append(locations_dict[string])
            found = True
            break
    if not found:
        raw_location_list.append('-')
raw['Location'] = raw_location_list            
    

Cleaning the process names to delete all information that was extracted into seperate attributes and which does not add any value

In [ ]:
for i in raw.index:
    for txt in ['market for ', 
    'market group for '] + location_strings:
        if raw['Process'][i].startswith(txt):
            raw['Process'][i] = raw['Process'][i].replace(txt, '')
        if raw['Process'][i].endswith(txt):
            raw['Process'][i] = raw['Process'][i].replace(txt, '')

Adding a second name column that serves as a shorthand notation for the full name

In [ ]:
process_shorthand_list = []
for i in range(len(raw)):
    process_shorthand_list.append(raw.iloc[i].Process.partition(',')[0]) #selecting part of the name up to first comma character
raw['Process_shorthand'] = process_shorthand_list

In [ ]:
# raw.to_csv(loading_url.replace('rawExcel', 'CSVs').replace('xlsx', 'csv'), index=False) #uncomment to export pre-processed dataset to csv